In [1]:
import os
import re
import time
import urllib
import requests
from django.core.files import File
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
import json

In [2]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')

In [3]:
아파트 = []
오피스텔 = []


In [4]:
driver.get('https://www.dabangapp.com/room/5e992d31d9d3a15c88491696')

In [5]:
post_type = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/p/span')
post_type = post_type.get_attribute('innerText')

# 방 정보 설명
description = driver.find_elements_by_xpath("/html/body/div[1]/div/div[4]/div/div")
description = description[0].get_attribute("innerText")
description.replace("\n", "")

# 매물 형식
unrefined_salesform = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/div')
salesForm = unrefined_salesform[0].get_attribute("innerText")
salesForm = salesForm.replace('/', ' ')
salesForm = salesForm.replace('\n', '')
salesForm = salesForm.split()
salesType = salesForm[0]  # sales type


In [6]:
print(post_type, salesType)

아파트 월세
